In [1]:
import yfinance as yf

In [2]:
stock = yf.download('A500.MI',period='max')

[*********************100%%**********************]  1 of 1 completed


In [3]:
stock

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-03-22,38.529999,38.724998,38.275002,38.494999,38.494999,20477
2018-03-23,37.750000,38.049999,37.610001,37.730000,37.730000,45786
2018-03-26,37.490002,37.540001,36.965000,37.064999,37.064999,282015
2018-03-27,37.915001,38.165001,37.845001,37.924999,37.924999,53260
2018-03-28,37.520000,37.520000,37.080002,37.264999,37.264999,61077
...,...,...,...,...,...,...
2024-07-02,99.459999,99.760002,99.220001,99.720001,99.720001,1710
2024-07-03,100.029999,100.160004,99.629997,99.849998,99.849998,2331
2024-07-04,100.239998,100.279999,100.010002,100.000000,100.000000,1825


In [9]:
stock['2020-02':'2023-01'].index[-1]

Timestamp('2023-01-31 00:00:00')

In [10]:
stock_today = yf.download('A500.MI', period='1d')

[*********************100%%**********************]  1 of 1 completed


In [11]:
stock_today

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-08,100.389999,100.540001,100.269997,100.510002,100.510002,5979


In [14]:
yf.download('A500.MI', period='1d', interval='1m')

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-07-09 09:04:00+02:00,100.739998,100.739998,100.739998,100.739998,100.739998,0
2024-07-09 09:11:00+02:00,100.669998,100.669998,100.669998,100.669998,100.669998,3
2024-07-09 09:35:00+02:00,100.800003,100.800003,100.800003,100.800003,100.800003,10
2024-07-09 09:45:00+02:00,100.839996,100.839996,100.839996,100.839996,100.839996,284
2024-07-09 09:53:00+02:00,100.820000,100.820000,100.820000,100.820000,100.820000,7
2024-07-09 10:01:00+02:00,100.900002,100.900002,100.900002,100.900002,100.900002,32
2024-07-09 10:08:00+02:00,100.900002,100.900002,100.900002,100.900002,100.900002,2
2024-07-09 10:19:00+02:00,100.870003,100.870003,100.870003,100.870003,100.870003,0


In [15]:
stock = yf.Ticker('A500.MI')

In [16]:
stock

yfinance.Ticker object <A500.MI>

In [17]:
stock.basic_info

lazy-loading dict with keys = ['currency', 'dayHigh', 'dayLow', 'exchange', 'fiftyDayAverage', 'lastPrice', 'lastVolume', 'marketCap', 'open', 'previousClose', 'quoteType', 'regularMarketPreviousClose', 'shares', 'tenDayAverageVolume', 'threeMonthAverageVolume', 'timezone', 'twoHundredDayAverage', 'yearChange', 'yearHigh', 'yearLow']

In [24]:
stock.historical_data()

AttributeError: 'Ticker' object has no attribute 'historical_data'

In [22]:
stock.get_info()['symbol']

'A500.MI'